In [8]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time

def record_object_es23(uuid_list, con_key, schema_name, store_frame='raw', add_pc_wfr=False):    
    """starting from a single uuid, tracks all linked items,
    keeps a list of uuids, and dictionary of items for each schema in the given store_frame"""
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    if add_pc_wfr:
        add_from_embedded = {'file_fastq': ['workflow_run_inputs', 'workflow_run_outputs'],
                             'file_processed': ['workflow_run_inputs', 'workflow_run_outputs']
                             }
    else:
        add_from_embedded = {}
        
    store={}
    item_uuids=[]
    
    while uuid_list:
        #find schema name, store as obj_key, create empty list if missing in store
        ES_items = ff_utils.get_es_metadata(uuid_list, key=con_key)
        uuids_to_check = []
        for ES_item in ES_items:
            uuid = ES_item['uuid']
            object_resp = ES_item['object']
            obj_type = object_resp['@type'][0]
            obj_key = schema_name[obj_type]
            if obj_key not in store:
                store[obj_key] = []
            raw_resp = ES_item['properties']
            raw_resp['uuid'] = uuid
            # add raw frame to store and uuid to list
            if uuid not in item_uuids:
                if store_frame == 'object':
                    store[obj_key].append(object_resp)
                else:
                    store[obj_key].append(raw_resp)
                item_uuids.append(uuid)
            # this case should not happen actually
            else:
                print('Problem encountered - skipped - check')
                continue

            #get linked items from es
            for key in ES_item['links']:
                uuids_to_check.extend(ES_item['links'][key])

            # check if any field from the embedded frame is required
            add_fields = add_from_embedded.get(obj_key)
            if add_fields:
                emb_resp = ES_item['embedded']
                for a_field in add_fields:
                    field_val = emb_resp.get(a_field)
                    if field_val:
                        #turn it into string
                        field_val = str(field_val)
                        # check if any of embedded uuids is in the field value
                        for a_uuid in ES_item['embedded_uuids']:
                            if a_uuid in field_val:
                                uuids_to_check.append(a_uuid)
        # get uniques
        uuids_to_check = list(set(uuids_to_check))
        uuid_list = []
        for an_uuid in uuids_to_check:
            if an_uuid not in item_uuids:
                uuid_list.append(an_uuid)
        print(len(uuid_list))
    
    return store, item_uuids

time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = ['69b42ba9-4333-48d7-bb52-75e33572e777']

store, uuids = record_object_es23(set_to_release, my_auth, schema_name, store_frame='raw', add_pc_wfr=False)
print()
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)


9
11
11
3
5
7
1
0

46
46
0.08867926597595215


In [10]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = '69b42ba9-4333-48d7-bb52-75e33572e777'
store2, uuids2 = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store={}, item_uuids=[])
print(len(uuids2))
print(len(list(set(uuids2))))
time2 = time.time()
print((time2-time1)/60)


48
48
0.3442159136136373


In [3]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time

def record_object_es22(uuid_list, con_key, schema_name, store_frame='raw', add_pc_wfr=False, store={}, item_uuids=[]):
    if not uuid_list:
        return store, item_uuids
    
    """starting from a single uuid, tracks all linked items,
    keeps a list of uuids, and dictionary of items for each schema in the given store_frame"""
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    if add_pc_wfr:
        add_from_embedded = {'file_fastq': ['workflow_run_inputs', 'workflow_run_outputs'],
                             'file_processed': ['workflow_run_inputs', 'workflow_run_outputs']
                             }
    else:
        add_from_embedded = {}

    #find schema name, store as obj_key, create empty list if missing in store
    ES_items = ff_utils.get_es_metadata(uuid_list, key=con_key)
    uuids_to_check = []
    for ES_item in ES_items:
        uuid = ES_item['uuid']
        object_resp = ES_item['object']
        obj_type = object_resp['@type'][0]
        obj_key = schema_name[obj_type]
        if obj_key not in store:
            store[obj_key] = []
        raw_resp = ES_item['properties']
        raw_resp['uuid'] = uuid
        # add raw frame to store and uuid to list
        if uuid not in item_uuids:
            if store_frame == 'object':
                store[obj_key].append(object_resp)
            else:
                store[obj_key].append(raw_resp)
            item_uuids.append(uuid)
        # this case should not happen actually
        else:
            print('Problem encountered - skipped - check')
            continue

        #get linked items from es
        for key in ES_item['links']:
            uuids_to_check.extend(ES_item['links'][key])

        # check if any field from the embedded frame is required
        add_fields = add_from_embedded.get(obj_key)
        if add_fields:
            emb_resp = ES_item['embedded']
            for a_field in add_fields:
                field_val = emb_resp.get(a_field)
                if field_val:
                    #turn it into string
                    field_val = str(field_val)
                    # check if any of embedded uuids is in the field value
                    for a_uuid in ES_item['embedded_uuids']:
                        if a_uuid in field_val:
                            uuids_to_check.append(a_uuid)
    
    # get uniques
    uuids_to_check = list(set(uuids_to_check))
    uuids_to_pass = []
    for an_uuid in uuids_to_check:
        if an_uuid not in item_uuids:
            uuids_to_pass.append(an_uuid)
    print(len(uuids_to_pass))
    if not(uuids_to_pass):
        print(uuids_to_pass)
    store, item_uuids = record_object_es2(uuids_to_pass, con_key, schema_name, store_frame, add_pc_wfr, store, item_uuids)